In [4]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

from langchain_tavily import TavilySearch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

In [19]:
all_pdf_documents = []
dir_loader=DirectoryLoader(
    "../data/pdfs",
    glob="**/*.pdf", 
    loader_cls= PyMuPDFLoader, 
    use_multithreading=True,
    show_progress=True

)

all_processed_docs = []

pdf_documents=dir_loader.lazy_load()

for doc in dir_loader.lazy_load():   
    doc.metadata["source_type"] = "Authorotative"
    doc.metadata["priority"] = "1"
    all_pdf_documents.append(doc)

 


100%|██████████| 4/4 [00:09<00:00,  2.35s/it]


In [20]:
load_dotenv()
os.getenv('TAVILY_API_KEY')

# Instantiate using the new name
web_search_tool = TavilySearch(
    max_results=3, 
    search_depth="advanced", 
    include_domains=["who.int", "cdc.gov", "mayoclinic.org"] 
)


try:
    results = web_search_tool.invoke({"query": "current 2025 health alerts"})
    print("Search Successful!")
    print(results)
except Exception as e:
    print(f"Error: {e}")


Search Successful!
{'query': 'current 2025 health alerts', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.cdc.gov/respiratory-viruses/data/index.html', 'title': 'Respiratory Illnesses Data Channel', 'content': 'Due to the Christmas holiday, the weekly respiratory virus data and summaries will not be updated on Friday, December 26, 2025. Data updates will resume on Tuesday, December 30, 2025.\n\n## What to know\n\nRI Alert Graphic Low\n\n## Your community snapshot\n\n### Keep in mind\n\n## Weekly national summary\n\n## Season Outlook [...] CDC continues to expect that the fall and winter respiratory disease season in the United States will likely have a similar number of combined peak hospitalizations due to COVID-19, influenza, and RSV compared to last season. Read more: 2025-2026 Respiratory Disease Season Outlook - December Update | CDC\n\n## Protect yourself and your community\n\n### Looking for guidance?\n\n## Continue exploring these da

In [21]:
def split_documents(pdf_documents,chunk_size=1000,chunk_overlap=200):

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(pdf_documents)
    #print(f"Split {len(pdf_documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [22]:
chunks=split_documents(all_pdf_documents)
chunks


Example chunk:
Content: i 
 
National Health 
Mission (NHM) 
Manual for District- Level Functionaries 
 
2017...
Metadata: {'producer': 'Foxit PhantomPDF Printer Version 6.0.4.1129', 'creator': '', 'creationdate': '2017-09-25T12:36:34+05:30', 'source': '../data/pdfs/National Health Mission.pdf', 'file_path': '../data/pdfs/National Health Mission.pdf', 'total_pages': 56, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2017-09-25T12:36:44+05:30', 'trapped': '', 'modDate': "D:20170925123644+05'30'", 'creationDate': "D:20170925123634+05'30'", 'page': 0, 'source_type': 'Authorotative', 'priority': '1'}


[Document(metadata={'producer': 'Foxit PhantomPDF Printer Version 6.0.4.1129', 'creator': '', 'creationdate': '2017-09-25T12:36:34+05:30', 'source': '../data/pdfs/National Health Mission.pdf', 'file_path': '../data/pdfs/National Health Mission.pdf', 'total_pages': 56, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2017-09-25T12:36:44+05:30', 'trapped': '', 'modDate': "D:20170925123644+05'30'", 'creationDate': "D:20170925123634+05'30'", 'page': 0, 'source_type': 'Authorotative', 'priority': '1'}, page_content='i \n \nNational Health \nMission (NHM) \nManual for District- Level Functionaries \n \n2017'),
 Document(metadata={'producer': 'Foxit PhantomPDF Printer Version 6.0.4.1129', 'creator': '', 'creationdate': '2017-09-25T12:36:34+05:30', 'source': '../data/pdfs/National Health Mission.pdf', 'file_path': '../data/pdfs/National Health Mission.pdf', 'total_pages': 56, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': ''

In [27]:
import numpy as np
from sentence_transformers import SentenceTransformer
import sys
from pydantic_settings import BaseSettings
import pydantic

pydantic.BaseSettings = BaseSettings

import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

ValidationError: 5 validation errors for Settings
clickhouse_host
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
clickhouse_port
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
chroma_server_host
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
chroma_server_http_port
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
chroma_server_grpc_port
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/str

In [23]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Error loading model all-MiniLM-L6-v2: name 'SentenceTransformer' is not defined


NameError: name 'SentenceTransformer' is not defined